<a href="https://colab.research.google.com/github/deepesh321/Machine_translation/blob/master/Machine_translation_encode_decode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
BASE_DIR = '/content/drive/My Drive/'

In [0]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

Using TensorFlow backend.


In [0]:
lines=pd.read_table(BASE_DIR+'mar.txt',names=['eng','mar','other'])

In [0]:
lines.sample(10)

,eng,mar,other
6897,He made me a cake.,त्याने माझ्यासाठी केक बनवला.,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
6043,The cat ran away.,मांजर पळून गेली.,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
17227,My sister is a teacher.,माझी बहीण शिक्षिका आहे.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
14424,Did you get any sleep?,झोप लागली का?,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
21683,He studies hard every day.,तो दररोज मेहनतीने अभ्यास करतो.,CC-BY 2.0 (France) Attribution: tatoeba.org #8...
36531,I had never seen that kind of fish until then.,तोपर्यंत मी तसला मासा कधीच पाहिला नव्हता.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
28036,They have one thing in common.,त्यांच्यात एक गोष्ट समान आहे.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
35566,"If you're going to the party, let me know.",पार्टीला जाणार असशील तर मला कळव.,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
20393,I was born in that house.,मी त्या घरात जन्माला आले.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
23998,The street is full of cars.,रस्ता गाड्यांनी भरलेला आहे.,CC-BY 2.0 (France) Attribution: tatoeba.org #4...


In [0]:
lines.eng=lines.eng.apply(lambda x:x.lower()) #lowercase all the characters
lines.mar=lines.mar.apply(lambda x:x.lower())

In [0]:
lines.eng=lines.eng.apply(lambda x: re.sub("'",'',x)) #removing quotes
lines.mar=lines.mar.apply(lambda x: re.sub("'",'',x))

In [0]:
exclude=set(string.punctuation)  # special character set
# removing special character
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.mar=lines.mar.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [0]:
#removing digits
remove_digits=str.maketrans('','',digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
lines.mar=lines.mar.apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

In [0]:
# Remove extra spaces
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.mar=lines.mar.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.mar=lines.mar.apply(lambda x: re.sub(" +", " ", x))

In [0]:
# Add start and end tokens to target sequences
lines.mar = lines.mar.apply(lambda x : 'START_ '+ x + ' _END')

In [0]:
lines.sample(10)

,eng,mar,other
971,it was mine,START_ माझा होता _END,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
23401,i am still a stranger here,START_ मी इथे अजूनही परकाच आहे _END,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
14385,can we change history,START_ आपण इतिहास बदलू शकतो का _END,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
19775,beer is sold by the pint,START_ बीअरला पाइन्टप्रमाणे विकलं जातं _END,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
25929,why did you buy the flowers,START_ तू ती फुलं कुठे विकत घेतलीस _END,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
4462,open the bottle,START_ बाटली उघड _END,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
21424,all knowledge is not good,START_ सर्वच ज्ञान काय चांगलं नसतं _END,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
149,i phoned,START_ मी फोन केला _END,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
29702,i have watched all of her films,START_ मी तिचे सगळेच चित्रपट पाहिले आहेत _END,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
12825,i arrived last night,START_ मी काल रात्री पोचले _END,CC-BY 2.0 (France) Attribution: tatoeba.org #3...


In [0]:
lines.drop(['other'], axis = 1) 

,eng,mar
0,go,START_ जा _END
1,run,START_ पळ _END
2,run,START_ धाव _END
3,run,START_ पळा _END
4,run,START_ धावा _END
...,...,...
38003,just saying you dont like fish because of the ...,START_ हड्डींमुळे मासे आवडत नाही असं म्हणणं हे...
38004,the japanese parliament today officially elect...,START_ आज जपानी संसदेने अधिकृतरित्या र्‍यौतारौ...
38005,tom tried to sell his old vcr instead of throw...,START_ टॉमने त्याचा जुना व्हीसीआर फेकून टाकण्य...
38006,you cant view flash content on an ipad however...,START_ आयपॅडवर फ्लॅश आशय बघता येत नाही पण तुम्...


In [0]:
# Vocabulary of English
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

# Vocabulary of French 
all_marathi_words=set()
for mar in lines.mar:
    for word in mar.split():
        if word not in all_marathi_words:
            all_marathi_words.add(word)

In [0]:
# Max Length of source sequence
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
max_length_src

34

In [0]:
# Max Length of target sequence
lenght_list=[]
for l in lines.mar:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
max_length_tar

37

In [0]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_marathi_words))
num_encoder_tokens = len(all_eng_words)+1
num_decoder_tokens = len(all_marathi_words)
num_encoder_tokens, num_decoder_tokens

(5559, 13271)

In [0]:
num_decoder_tokens += 1 # For zero padding
num_decoder_tokens

13272

In [0]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])
#print(input_token_index)

In [0]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())
#print(reverse_input_char_index)

In [0]:
lines = shuffle(lines)
lines.drop(['other'],axis=1)

,eng,mar
21532,do you know where he went,START_ तो कुठे गेला तुम्हाला माहीत आहे का _END
19497,were you born in boston,START_ तुमचा जन्म बॉस्टनमध्ये झालेला का _END
17198,my car battery is dead,START_ माझ्या गाडीची बॅटरी बंद पडली आहे _END
13972,we meet once a month,START_ आपण महिन्यातून एकदा भेटतो _END
10456,didnt tom tell you,START_ टॉमने तुम्हाला सांगितलं नाही का _END
...,...,...
32122,learning english requires patience,START_ इंग्रजी शिकण्यासाठी धैर्य लागतं _END
27974,the children went out to play,START_ मुले बाहेर खेळायला गेली _END
20685,she didnt marry the man,START_ तिने त्या माणसाशी लग्न केलं नाही _END
34270,he said he had seen her a month before,START_ तो म्हणाला की त्याने तिला एक महिन्यापूर...


In [0]:
# Train - Test Split
X, y = lines.eng, lines.mar
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((34207,), (3801,))

In [0]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    #print('hello')
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            #print('hello')
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                #print(i)
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                    #print(input_token_index[word] )
                for t, word in enumerate(target_text.split()):
                    #print(t)
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            #print(encoder_input_data.shape,decoder_input_data.shape)
            #print(encoder_input_data[0])
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [0]:
#encoder
import tensorflow as tf
latent_dim=50
encoder_inputs = Input(shape=(None,))
enc_emb=Embedding(num_encoder_tokens,latent_dim,mask_zero=True)(encoder_inputs)
encoder_lstm=LSTM(latent_dim,return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states=[state_h,state_c]




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [0]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [0]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 50
generate_batch(X_train,y_train,batch_size=batch_size)

<generator object generate_batch at 0x7ffab0bb6e60>

In [0]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)




Epoch 1/50





267/267 [==============================] - 952s 4s/step - loss: 6.4608 - acc: 0.1784 - val_loss: 5.8014 - val_acc: 0.1984
Epoch 2/50
267/267 [==============================] - 926s 3s/step - loss: 5.5734 - acc: 0.2092 - val_loss: 5.4585 - val_acc: 0.2243
Epoch 3/50
267/267 [==============================] - 933s 3s/step - loss: 5.2761 - acc: 0.2358 - val_loss: 5.2517 - val_acc: 0.2502
Epoch 4/50
267/267 [==============================] - 936s 4s/step - loss: 5.0473 - acc: 0.2643 - val_loss: 5.0464 - val_acc: 0.2736
Epoch 5/50
105/267 [==========>...................] - ETA: 9:08 - loss: 4.9037 - acc: 0.2751

In [0]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [0]:
Image(retina=True, filename='encoder_model.png')

In [0]:
Image(retina=True, filename='decoder_model.png')

In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [0]:
val_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [0]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

In [0]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])